<h3>Installing Dependencies</h3>

In [14]:
import os, sys

import matplotlib.pyplot as plt 

import numpy as np 
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

from sklearn.linear_model import LinearRegression

<p> Bui

In [15]:
train_data = pd.read_csv('blogData_train.csv')

In [ ]:
features = [['']]